## 🏷️ 복원된 리뷰를 바탕을 난독화된 리뷰 문장 단위로 분할

In [1]:
import pandas as pd

In [ ]:
test_df = pd.read_csv('../data/test.csv', encoding='utf-8-sig') # 난독화된 리뷰
post_df = pd.read_csv('../submissions/submission_postprocess.csv', encoding='utf-8-sig') # 학습된 모델이 복원된 리뷰를 후처리한 리뷰 (난독화된 리뷰와 길이 일치)

In [3]:
post_df.head()

ID                                             output
0  TEST_0000  너무너무 만족스러운 호텔이에요. 부산에 오면 꼭 추천하고 싶은 곳이에요. 최고입니다...
1  TEST_0001  프론트가 없고, 조식도 없으며, 일반 입주민들이 샤있샤윔엡 있어 호텔처럼 관리가 잘...
2  TEST_0002  진짜 불친절해요. 살면서 머물렀던 호텔 중에 최악이었습니다. 직원인지 사장인지 체크...
3  TEST_0003  뷰 맛집~~ 그런데 방음이 미흡하네요. 층간 소음과 발코니가 이중창이 아니라서 밤에...
4  TEST_0004  방 상태는 진짜 폐허 직전인데 전망은 좋아요. 보일러가 아주 찬찬하게 돌아서 추웠어...

## 🏷️ 복원된 리뷰 문장 단위로 분할

In [4]:
from kss import split_sentences

post_df['sentences'] = post_df["output"].apply(lambda x: split_sentences(x))

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



In [5]:
post_df['sentences'].head()

0    [너무너무 만족스러운 호텔이에요., 부산에 오면 꼭 추천하고 싶은 곳이에요., 최고...
1    [프론트가 없고, 조식도 없으며, 일반 입주민들이 샤있샤윔엡 있어 호텔처럼 관리가 ...
2    [진짜 불친절해요., 살면서 머물렀던 호텔 중에 최악이었습니다., 직원인지 사장인지...
3    [뷰 맛집~~ 그런데 방음이 미흡하네요., 층간 소음과 발코니가 이중창이 아니라서 ...
4    [방 상태는 진짜 폐허 직전인데 전망은 좋아요., 보일러가 아주 찬찬하게 돌아서 추...
Name: sentences, dtype: object

## 🏷️ 복원된 리뷰 문장을 바탕으로 난독화된 리뷰 문장 단위로 분할

In [ ]:
test_sentences = {"ID" : [], "input": [], "start_idx": [], "end_idx": []}

for idx, row in post_df.iterrows():
    print(f"{idx}")
    for i, sent in enumerate(row['sentences']):
        start_idx = row['output'].index(sent)
        end_idx = start_idx + len(sent)
        mapping_sent = test_df['input'][idx][start_idx:end_idx]
        print(mapping_sent)

        test_sentences['ID'].append(row['ID'])
        test_sentences['input'].append(mapping_sent)
        test_sentences['start_idx'].append(start_idx)
        test_sentences['end_idx'].append(end_idx)

    print(f"{len(row['sentences'])} sentences")
    print()

In [8]:
test_sentences_df = pd.DataFrame(test_sentences)
test_sentences_df.head()

ID                                              input  start_idx  \
0  TEST_0000                                  녀뮨넒뭅 만죡숭러윤 효템뤼에오.          0   
1  TEST_0000                             푸싸눼 옰면 콕 츄쩐학꼬 싶은 콧쉰웨오.         18   
2  TEST_0000                                          췌꾜윕뉘댜! ㅎㅎ         41   
3  TEST_0000                                     당음웨 또 옭 컷 갗았요.         51   
4  TEST_0001  풀룐투갸 엎코, 좀식또 업읍머, 윌뱐 잎츔민든릿 샤있샤윔엡 위썬 호뗄첨렴 관뤽갉 찰...          0   

   end_idx  
0       17  
1       40  
2       50  
3       65  
4       58

In [9]:
len(test_sentences_df)

8365

## 🏷️ 문장 단위로 분리된 난독화된 리뷰 저장

In [ ]:
test_sentences_df.to_csv('../data/test_sentences.csv', index=False, encoding='utf-8-sig')